# 画像分類

## *ライブラリのインポート*

In [33]:
from tensorflow.keras import backend

from keras.utils import np_utils
from tensorflow.keras.models import Sequential
from tensorflow.keras import regularizers
from tensorflow.keras.layers import MaxPooling2D, GlobalAveragePooling2D, Activation, Conv2D, Input, BatchNormalization, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import SGD, Adadelta, Adagrad, Adam, Adamax, RMSprop, Nadam
from sklearn.model_selection import train_test_split
from PIL import Image, ImageOps

import numpy as np
import glob
import matplotlib.pyplot as plt
import time
import os

## 関数定義

In [34]:
def compare_TV(history):
    import matplotlib.pyplot as plt

    # Setting Parameters
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(len(acc))

    # 1) Accracy Plt
    plt.plot(epochs, acc, 'bo' ,label = 'training acc')
    plt.plot(epochs, val_acc, 'b' , label= 'validation acc')
    plt.title('Training and Validation acc')
    plt.legend()

    plt.figure()

    # 2) Loss Plt
    plt.plot(epochs, loss, 'bo' ,label = 'training loss')
    plt.plot(epochs, val_loss, 'b' , label= 'validation loss')
    plt.title('Training and Validation loss')
    plt.legend()

    plt.show()

## グレースケール

In [35]:
def img_convert(file, color, x, y):
    
    img = Image.open(file)
    if img.mode != "RGB":
        img = img.convert("RGB") # any format to RGB

    # 回転
    if img.size[0] < img.size[1]:
        img = img.rotate(90, expand=True)

    # グレースケール変換
    if color != "RGB":
        rgb = np.array(img, dtype="float32");
        rgbL = pow(rgb/255.0, 2.2)
        r, g, b = rgbL[:,:,0], rgbL[:,:,1], rgbL[:,:,2]
        grayL = 0.299 * r + 0.587 * g + 0.114 * b  # BT.601
        gray = pow(grayL, 1.0/2.2)*255
        img = Image.fromarray(gray.astype("uint8"))
        
    img = img.convert(color).resize((x, y))
    img = np.asarray(img)
    return img

## グローバル変数設定

In [37]:
# RGB, L, 1
color = "RGB"
channels = 1
if color is "RGB":
    channels = 3

## Google Driveのマウント

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
img_rows, img_cols = 96, 128 

train_target = "/content/drive/My Drive/ensyu2/animals"

train_folder = ['cat', 'dog']
print(train_folder)

dense_size = len(train_folder)
print(dense_size)

## 訓練データ読み込み

In [ ]:
X = []
Y = []
for index, name in enumerate(train_folder):
    #パス変更
    dir = train_target + "/" + name
    files = glob.glob(dir + "/*.jpg")
    for i, file in enumerate(files):
        data = img_convert(file, color, img_cols, img_rows)
        X.append(data)
        Y.append(index)  
    print(len(Y))
    
X = np.array(X)
Y = np.array(Y)
X = X.astype('float32') / 255

Y = np_utils.to_categorical(Y, dense_size)

print('Training Data Read Done')

In [ ]:
# L:グレースケール、 1:白黒の対応
if color is not "RGB":
    X = X.reshape(X.shape[0], X.shape[1], X.shape[2], channels)
print(X.shape)
print(dense_size)

## モデルの定義

In [ ]:
#Start Neural Network
model = Sequential()

#convolution 1st layer
model.add(Conv2D(32, (3, 3), padding='same', input_shape=X.shape[1:]))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

#convolution 2st layer
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

#convolution 3st layer
model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

#convolution 4st layer
model.add(Conv2D(256, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

#Fully connected 1st layer
model.add(Flatten())
#model.add(GlobalAveragePooling2D())

model.add(Dense(1024, kernel_regularizer=regularizers.l2(0.01)))
#model.add(Dense(1024))
model.add(Activation('relu'))
model.add(Dropout(0.4))

model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.1))

model.add(Dense(64))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(Dense(dense_size))
model.add(Activation('softmax'))

model.summary()

## モデル学習
https://keras.io/ja/optimizers/   
Adadelta以外にもSGD, Adagrad, Adam, Adamax, RMSprop, Nadam

In [ ]:
history = {}
#optimizers = Adadelta(lr=1.0, rho=0.95, epsilon=None, decay=0.0)
#optimizers = SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=False)
#optimizers = Adagrad(lr=0.01, epsilon=None, decay=0.0)
#optimizers = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
optimizers = Adamax(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0)
#optimizers = RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0)
#optimizers = Nadam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004)
epochs = 10

model.compile(loss='categorical_crossentropy', optimizer=optimizers, metrics=['acc'])
print(len(X))
print(len(Y))
history = model.fit(X, Y, validation_split=0.20, batch_size=8, shuffle=True, verbose=1, epochs=epochs)

1 train_dataに対してaccuracyとlossを計算する  
2 lossを使ってモデルを更新する  
3 test_data分1と2を繰り返す  
4 学習されたモデルに対してvalidation_dataで検証する(この際出たlossでモデルの更新は行わない)  
5 上記の1～4の流れを1epochとして何度も繰り返す  
6 最後にできたモデルに対してtest_dataで最終的な性能を測る  

In [ ]:
score = model.evaluate(X, Y, verbose=1)
print('正解率=', score[1], '損失率=', score[0])

compare_TV(history)

## 評価データ読込・正答率確認

In [ ]:
#パス変更
test_target = "/content/drive/My Drive/ensyu2/test"
test_folder = ['cat', 'dog']
print(test_folder)

X_test = []
Y_test = []
f_test = []
for index, name in enumerate(test_folder):
    dir = test_target + "/" + name
    files = glob.glob(dir + "/*.jpg")
    files.sort()
    for i, file in enumerate(files):
        img = img_convert(file, color, img_cols, img_rows)
        data = img.reshape(img.shape[0], img.shape[1], channels)
        X_test.append(data)
        Y_test.append(index)
        f_test.append(file)

X_test = np.array(X_test)
X_test = X_test.astype('float32') / 255
category = model.predict_classes(X_test)

print('finish')

In [ ]:
ok_num = 0
ok_list = [[0,0], [0,0]]
percet = [0.0, 0.0]

for index, iy in enumerate(Y_test):
    print(index, iy, category[index], f_test[index])
    if iy == category[index]:
        ok_num = ok_num + 1
        ok_list[iy][0] = ok_list[iy][0] + 1
    ok_list[iy][1] = ok_list[iy][1] + 1

print(ok_list)
str = ''
for item in ok_list:
    if 0 == item[1]:
        str += '0.0,'
    else:
        str += '{:.2f}, '.format(item[0]/item[1])
print(str)
print(category)
print(ok_num, "/", index+1, "=", ok_num/(index+1))

# モデル保存

In [ ]:
model_path = '/content/drive/My Drive/ensyu2/wk_model/ai_animal'
model_json_str = model.to_json()

open(model_path + '.json', 'w').write(model_json_str)
model.save_weights(model_path + '.h5');

# モデル読込・確認

In [ ]:
import tensorflow as tf 
from keras.models import model_from_json

def model_read(path):
    model_arc_str = open(path + ".json").read()
    model = tf.keras.models.model_from_json(model_arc_str)

    # モデル構成の確認
    #model.summary()

    # モデルの重みを得る
    model.load_weights(path + ".h5")
    return model